# Bitcoin trading strategy dashboard

## Checking the data source

We will be reading data from the file `aggr.csv` which was made available by the creators of the strategy. Let's load it into a `pandas` dataframe and look at the columns available to us for use:

In [2]:
import pandas as pd

df = pd.read_csv('aggr.csv', parse_dates=['Entry time'])
df.head()

,Number,Trade type,Entry time,Exposure,Entry balance,Exit balance,Profit,Pnl (incl fees),Exchange,Margin,BTC Price
0,505,Short,2019-09-16 12:30:00,2 days 11 hours 30 minutes,1499.70810,1497.45854,-2.249562,-0.15,Bitmex,1,10262.5
1,504,Long,2019-09-15 22:00:00,14 hours 30 minutes,1523.08014,1499.23112,-23.849025,-1.57,Bitmex,1,10304.4
2,503,Short,2019-09-09 08:00:00,6 days 14 hours,1548.75311,1522.55620,-26.196910,-1.69,Bitmex,1,10310.0
3,502,Long,2019-09-07 15:00:00,1 day 17 hours,1594.40783,1547.82139,-46.586446,-2.92,Bitmex,1,10484.3
4,501,Short,2019-09-06 20:00:00,19 hours,1604.98017,1594.19207,-10.788103,-0.67,Bitmex,1,10306.9


The available data columns are as follows:

1. **Number**: The serial number of the trade
2. **Trade type**: Whether the trade was a "Long" or a "Short"
3. **Entry time**: The entry time of the trade
4. **Exposure**: The length of the trade before exiting
5. **Entry balance**: BTC balance at entry
6. **Exit balance**: BTC balance at exit
7. **Profit**: Profit (in BTC) for the trade (can be positive or negative)
8. **Pnl (incl fees)**: Profit or loss of trade in %
9. **Exchange**: Exchange the trade was executed on
10. **Margin**: Margin or leverage used in the trade (hereforth, margin/leverage will be used interchangably)
11. **BTC Price**: Closing price of BTC on entry date

### Ideal end state of the dashboard


![screen1](img2.png "Dashboard Screenshot")
![screen2](img1.png "Dashboard Screenshot")

## Setting up the Dash app

Let's get started building the above design. Copy the following into an `app.py` file:

```
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

df = pd.read_csv('aggr.csv', parse_dates=['Entry time'])

app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css', 'https://codepen.io/uditagarwal/pen/YzKbqyV.css'])

if __name__ == "__main__":
    app.run_server(debug=True)
```

We will start by adding the selectors to the dashboard. The first part of the layout will be the title of our analysis. The class names of the divs are based on the styling information specified in the CSS file which is imported during setup of the app. Add the following to your `app.py` file:

```
app.layout = html.Div(children=[
    html.Div(
            children=[
                html.H2(children="Bitcoin Leveraged Trading Backtest Analysis", className='h2-title'),
            ],
            className='study-browser-banner row'
    )]
)
```

## Adding selectors

```PYTHON
app.layout = html.Div(children=[
    html.Div(
            children=[
                html.H2(children="Bitcoin Leveraged Trading Backtest Analysis", className='h2-title'),
            ],
            className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="padding row",
                        children=[
                            html.Div(
                                className="two columns card",
                                children=[
                                    html.H6("Select Exchange",),
                                    dcc.RadioItems(
                                        id="exchange-select",
                                        options=[
                                            {'label': label, 'value': label} for label in df['Exchange'].unique()
                                        ],
                                        value='Bitmex',
                                        labelStyle={'display': 'inline-block'}
                                    )
                                ]
                            ),
                            # Leverage Selector
                            html.Div(
                                className="two columns card",
                                children=[
                                    html.H6("Select Leverage"),
                                    dcc.RadioItems(
                                        id="leverage-select",
                                        options=[
                                            {'label': str(label), 'value': str(label)} for label in df['Margin'].unique()
                                        ],
                                        value='1',
                                        labelStyle={'display': 'inline-block'}
                                    ),
                                ]
                            ),
                            html.Div(
                                className="two columns card",
                                children=[
                                    html.H6("Select a Date Range"),
                                    dcc.DatePickerRange(
                                        id="date-range-select",
                                        display_format="MMM YY",
                                        start_date=df['Entry time'].min(),
                                        end_date=df['Entry time'].max()
                                    ),
                                ]
                            ),
  
                        ]
                )
        ])
    ])        
])
```

### Add a callback function

Add a callback function which updates the start and end date values of the date range selector based on the exchange selected (since each exchange contains data on different time periods). Please type code for the callback in the cell below in addition to having it in your `app.py` file.

In [1]:
@app.callback(
    [dash.dependencies.Output(component_id='date-range-select', component_property='start_date'),
     dash.dependencies.Output(component_id='date-range-select', component_property='end_date')],
    [
        dash.dependencies.Input(component_id='exchange-select', component_property='value')
    ]
)
def update_date_exchange(value):
    t=df[df['Exchange']==value]['Entry time'].apply(lambda x: x.strftime('%Y-%m-%d'))
    start_date = t.min()
    end_date = t.max()
    return [start_date,end_date]

### Create a function to filter information

To ease the construction of all our callback functions, create a helper function `filter_df()` that will be re-used through the app. Our DataFrame will need to be filtered frequently based on the selection for `Exchange`, `Margin`, `start_date`, and `end_date`. Please type code for the function in the cell below in addition to having it in your `app.py` file.

```PYTHON

def filter_df(df,exchange, margin, start_date,end_date):
    filtered_df=df[(df['Exchange']==exchange) & (df['Margin']==margin) & (df['Entry time']>=start_date) & (df['Entry time']<=end_date)]
    
    return filtered_df
```

## Adding a data table to our plot

Recall that we wish to occasionally analyze specific trades over a given time period. A data table containing information per trade is suited for this.  A reference guide for data tables in Dash is here: https://dash.plot.ly/datatable.

To add a data table, we need to import the `dash_table` library. Let's add the table to our layout by setting a new div and placing it beneath the monthly plot:

In [ ]:
import dash_table

html.Div(
        className="padding row",
        children=[
            html.Div(
                className="six columns card",
                children=[
                    dash_table.DataTable(
                        id='table',
                        columns=[
                            {'name': 'Number', 'id': 'Number'},
                            {'name': 'Trade type', 'id': 'Trade type'},
                            {'name': 'Exposure', 'id': 'Exposure'},
                            {'name': 'Entry balance', 'id': 'Entry balance'},
                            {'name': 'Exit balance', 'id': 'Exit balance'},
                            {'name': 'Pnl (incl fees)', 'id': 'Pnl (incl fees)'},
                        ],
                        style_cell={'width': '50px'},
                        style_table={
                            'maxHeight': '450px',
                            'overflowY': 'scroll'
                        },
                    )
                ]
            ),
        ]
    ),

We pass the columns as a list of dictionaries, with `name` and `id` as the keys. Since we will be plotting the trade type, exposure, entry balance, etc. we pass these as the columns to our data table. The styling of the data table is important because we only want the height of the table to be 450px so that it doesn't overflow on the dashboard. Setting the `style_table` attribute with `maxHeight` and `overflowY` allows us to control for this. The table data will be rendered using the following callback function:

In [ ]:
@app.callback(
    dash.dependencies.Output('table', 'data'),
    (
        dash.dependencies.Input('exchange-select', 'value'),
        dash.dependencies.Input('leverage-select', 'value'),
        dash.dependencies.Input('date-range-select', 'start_date'),
        dash.dependencies.Input('date-range-select', 'end_date'),
    )
)
def update_table(exchange, leverage, start_date, end_date):
    dff = filter_df(df, exchange, leverage, start_date, end_date)
    return dff.to_dict('records')

## Bar chart for visualizing trades

Let's now add the bar chart to help visualize our trades. This can help us see the distribution of long vs. short trades as well as the most profitable trades over the selected period. Let's add this to our layout next to the data table, with the ID `pnl-types`:

```

        html.Div(
                className="padding row",
                children=[
                    html.Div(
                        className="six columns card",
                        children=[
                            dash_table.DataTable(
                                id='table',
                                columns=[
                                    {'name': 'Number', 'id': 'Number'},
                                    {'name': 'Trade type', 'id': 'Trade type'},
                                    {'name': 'Exposure', 'id': 'Exposure'},
                                    {'name': 'Entry balance', 'id': 'Entry balance'},
                                    {'name': 'Exit balance', 'id': 'Exit balance'},
                                    {'name': 'Pnl (incl fees)', 'id': 'Pnl (incl fees)'},
                                ],
                                style_cell={'width': '50px'},
                                style_table={
                                    'maxHeight': '450px',
                                    'overflowY': 'scroll'
                                },
                            )
                        ]
                    ),
                    dcc.Graph(
                        id="pnl-types",
                        className="six columns card",
                        figure={}
                    )
                ]
            )

```

### Callback function for bar chart

We will add a callback function to load the data we are looking to visualize into a bar chart. The x-axis should be the entry time for that trade, and the y-axis should be the profit or loss for that trade (incl fees) in percentage terms. 

In [4]:
@app.callback(
    dash.dependencies.Output('pnl-types', 'figure'),
    (
        dash.dependencies.Input('exchange-select', 'value'),
        dash.dependencies.Input('leverage-select', 'value'),
        dash.dependencies.Input('date-range-select', 'start_date'),
        dash.dependencies.Input('date-range-select', 'end_date'),
    )
)
def update_barchart(exchange,leverage, start_date,end_date):
    dff = filter_df(df, exchange, leverage, start_date, end_date)
    trace1 = go.Bar(x=dff[dff['Trade type']=='Long']['Entry time'], y=dff[dff['Trade type']=='Long']['Pnl (incl fees)'], name='Long', )
    trace2 = go.Bar(x=dff[dff['Trade type']=='Short']['Entry time'], y=dff[dff['Trade type']=='Short']['Pnl (incl fees)'], name='Short', )
    
    
    return {
        'data': [trace1, trace2],
        'layout': go.Layout(title='Pnl vs Trade type',
                            colorway=["#EF963B", "#EF533B"], hovermode="closest")}

## Adding two last charts

We will now add our two last charts to the layout. One will track the overall value of the portfolio over time, and the other will plot the value of BTCUSD over that same period. This will help us visualize trends in BTC markets as well as the overall value of our balance. Let's add two charts to the layout at the bottom:

```
html.Div(
        className="padding row",
        children=[
            dcc.Graph(
                id="daily-btc",
                className="six columns card",
                figure={}
            ),
            dcc.Graph(
                id="balance",
                className="six columns card",
                figure={}
            )
        ]
    )
```

### Callback functions for the last two charts

One function returns a line chart of the price of BTC over the selected period, and he another one returns a line chart of the portfolio balance over the same period. 

In [5]:
@app.callback(
    dash.dependencies.Output('daily-btc', 'figure'),
    (
        dash.dependencies.Input('exchange-select', 'value'),
        dash.dependencies.Input('leverage-select', 'value'),
        dash.dependencies.Input('date-range-select', 'start_date'),
        dash.dependencies.Input('date-range-select', 'end_date'),
    )
)
def update_dailybtc(exchange,leverage, start_date,end_date):
    dff = filter_df(df, exchange, leverage, start_date, end_date)
    trace=go.Scatter(x=dff["Entry time"], y=dff['BTC Price'], name='BTC Price', mode='lines',
                                marker={'size': 8, "opacity": 0.6, "line": {'width': 0.5}}, )
    return {"data": [trace],
            "layout": go.Layout(title="Daily BTC Price", xaxis={"title": "Date"})}

@app.callback(
    dash.dependencies.Output('balance', 'figure'),
    (
        dash.dependencies.Input('exchange-select', 'value'),
        dash.dependencies.Input('leverage-select', 'value'),
        dash.dependencies.Input('date-range-select', 'start_date'),
        dash.dependencies.Input('date-range-select', 'end_date'),
    )
)
def update_balance(exchange,leverage, start_date,end_date):
    dff = filter_df(df, exchange, leverage, start_date, end_date)
    trace=go.Scatter(x=dff["Entry time"], y=dff['Entry balance'], name='Balance', mode='lines',
                                marker={'size': 8, "opacity": 0.6, "line": {'width': 0.5}}, )
    return {"data": [trace],
            "layout": go.Layout(title="Balance overtime", xaxis={"title": "Date"})}